In [1]:
import spacy
import re
import en_core_web_sm
import en_core_web_lg
import math

In [2]:
nlp_sm = en_core_web_sm.load()
nlp_sm.max_length = 2500000

nlp_lg = en_core_web_lg.load()
nlp_lg.max_length = 2500000

In [3]:
doc = nlp1(u"Apple is looking at buying U.K. startup for $1 billion. Bran Stark is a boy.")
for token in doc:
   print(token.lemma_, token.text) 
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

NameError: name 'nlp1' is not defined

In [4]:
def union_entities(listA, listB):
    result = listA
    for ent_b,y1,y2 in listB:
        overlap = False
        for ent_a,x1,x2 in listA:
            if x1 <= y2 and y1 <= x2: # overlap
                overlap = True
                if (y2-y1) > (x2-x2): # the largest entity is B
                    result.remove((ent_a,x1,x2))
                    result.append((ent_b,y1,y2))
                break
        if not overlap:
            result.append((ent_b,y1,y2))
    return result


def process_chapter(chapter):
    doc1 = nlp_sm(chapter)
    doc2 = nlp_lg(chapter)
    sm = []
    lg = []
    for ent in doc1.ents:
        if ent.label_ == 'PERSON':
            sm.append((ent.text, ent.start_char, ent.end_char))
    for ent in doc2.ents:
        if ent.label_ == 'PERSON':
            lg.append((ent.text, ent.start_char, ent.end_char))

    return union_entities(sm,lg)


def process_book(head, tail, book_number):
    filename = head + str(book_number) + tail
    with open(filename) as fp:
        book = fp.read()
    regex = r"\n{1,}([A-Z\s]+)\n{2,}"
    chunks = re.split(regex,book)
    
    entities = {}
    for index, chunk in enumerate(chunks):
        if chunk.isupper():
            #print(chunk.strip().title(),chunks[index+1][:100].strip())
            title = str(book_number) + '–' + str(math.floor(index/2)) + '-' + chunk.strip().title()
            chapter = chunks[index+1].strip().replace('\t','').replace('"','').replace('\n',' ').replace(' . . .','...')
            entities[title] = process_chapter(chapter)
    return entities

In [5]:
characters = {} #––––––––––––––––––––––––––––––––
for i in range (1,6):
    characters.update(process_book('books/GOT','.txt',i))
    print('book',i,'done')

book 1 done
book 2 done
book 3 done
book 4 done
book 5 done


In [6]:
characters['5–63-Victarion']

[('Grief', 1014, 1019),
 ('Iron Victory', 1024, 1036),
 ('Fingerdancer', 1104, 1116),
 ('Lamentation', 1206, 1217),
 ('Greyjoy', 2971, 2978),
 ('Vole', 3526, 3530),
 ('Aeron', 3942, 3947),
 ('Myr', 6369, 6372),
 ('Dove', 6378, 6382),
 ('Iron Wing', 7496, 7505),
 ('Kiss', 7533, 7537),
 ('Yunkai', 10280, 10286),
 ('Meereen', 10420, 10427),
 ('Seven Skulls', 10896, 10908),
 ('Yunkishman', 11692, 11702),
 ('Iron Victory', 14008, 14020),
 ('Qarth', 16108, 16113),
 ('Meereen', 17712, 17719),
 ('Euron', 19649, 19654),
 ('Moqorro', 218, 225),
 ('Ghiscari', 247, 255),
 ('Longwater Pyke', 257, 271),
 ('Victarion Greyjoy', 307, 324),
 ('Victarion', 536, 545),
 ('Sparrowhawk', 1082, 1093),
 ('Ghiscari', 1149, 1157),
 ('Ghozai', 1343, 1349),
 ('Ghiscari Dawn', 1480, 1493),
 ('Victarion', 1546, 1555),
 ('Yunkai', 1621, 1627),
 ('Ghiscari', 1705, 1713),
 ('Victarion Greyjoy', 1764, 1781),
 ('Moqorro', 1798, 1805),
 ('Meereen', 1885, 1892),
 ('Ghiscari', 1956, 1964),
 ('Victarion', 2007, 2016),
 ('Dae

In [7]:
import pickle
with open ('GOT-characters-raw.pickle','wb+') as blavlaz:
    pickle.dump(characters, blavlaz)